# Swift XRT Flux Density Conversion

This notebook demonstrates how to use `grb_pipeline` to convert Swift XRT flux density
measurements from a reference energy (e.g., 10 keV) to any desired energy using
the time-resolved photon index.

The conversion formula: F(E) = F(E₀) × (E₀/E)^(Γ-1)

Authors: Vikas Chand

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from astropy.io import ascii
from grb_pipeline.analysis.spectral import SpectralAnalyzer, XRTFluxDensity, XRTFluxDensityData

## 1. Read XRT Flux Density and Photon Index Data

In [ ]:
# Read the flux density and photon index data
data = XRTFluxDensity.read_data(
    flux_density_file='GRB201015A_Fluxdensity10.txt',
    photon_index_file='GRB201015A_phindex.txt',
    reference_energy=10.0  # keV
)

print(f"Number of data points: {len(data.time)}")
print(f"Reference energy: {data.reference_energy} keV")
print(f"Time range: [{data.time.min():.1f}, {data.time.max():.1f}] s")
print(f"Photon index range: [{data.photon_index.min():.2f}, {data.photon_index.max():.2f}]")

## 2. Convert Flux Density to 1 keV

Using the photon index at each epoch, we convert the @10 keV flux density to @1 keV.

In [ ]:
# Convert to 1 keV
target_energy = 1.0  # keV
flux_1keV, flux_1keV_err_pos, flux_1keV_err_neg = XRTFluxDensity.convert_to_energy(
    data, target_energy=target_energy
)

print(f"Converted {len(flux_1keV)} flux density points to {target_energy} keV")

## 3. Publication-Quality Plot

Two-panel figure showing the flux density at 1 keV (top) and photon index evolution (bottom).

In [ ]:
fig = plt.figure(figsize=(12, 10))
gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1], hspace=0)

# Top panel: Flux density at 1 keV
ax1 = fig.add_subplot(gs[0])
ax1.errorbar(
    data.time, flux_1keV,
    xerr=(-data.time_err_neg, data.time_err_pos),
    yerr=(flux_1keV_err_pos, flux_1keV_err_neg),
    ls='none', marker='.', color='k', elinewidth=1.5,
    capsize=0, markersize=6, alpha=0.8, label='@1 keV'
)
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_ylabel(r'Flux density (Jy)', fontsize=18)
ax1.legend(fontsize=14)
ax1.tick_params(direction='in', which='both', labelsize=14)
ax1.tick_params(labelbottom=False)

# Bottom panel: Photon index
ax2 = fig.add_subplot(gs[1], sharex=ax1)
ax2.errorbar(
    data.time, data.photon_index,
    xerr=(-data.time_err_neg, data.time_err_pos),
    yerr=(-data.photon_index_err_neg, data.photon_index_err_pos),
    ls='none', marker='.', color='k', elinewidth=1.5,
    capsize=0, markersize=6
)
ax2.set_xscale('log')
ax2.set_xlabel(r'Time since $T_0$ (s)', fontsize=18)
ax2.set_ylabel('Photon Index', fontsize=18)
ax2.set_ylim(0.8, 4.0)
ax2.tick_params(direction='in', which='both', labelsize=14)

plt.tight_layout()
plt.show()

## 4. Using the Low-Level API

You can also use `SpectralAnalyzer.flux_density_convert` directly for custom conversions.

In [ ]:
# Convert a single measurement
factor = SpectralAnalyzer.flux_density_convert(E0=10.0, E=1.0, photon_index=2.0)
print(f"Conversion factor from 10 keV to 1 keV (Gamma=2.0): {factor:.4f}")

# Convert to a different energy
factor_5keV = SpectralAnalyzer.flux_density_convert(E0=10.0, E=5.0, photon_index=2.0)
print(f"Conversion factor from 10 keV to 5 keV (Gamma=2.0): {factor_5keV:.4f}")

In [ ]:
# Save converted data to file
XRTFluxDensity.save_converted(data, target_energy=1.0, output_file='GRB201015A_XRT_Fd1keV.txt')
print("Saved converted flux density data to GRB201015A_XRT_Fd1keV.txt")